In [ ]:
import os
import csv
import numpy as np
from scipy.ndimage import label

irectory info

In [ ]:
folder_path = "./images"
output_file = "1_features.csv" 
studentNo = "1"

ue to being unable to code the eyes module, they are hard coded here.

In [ ]:
label_to_eye_count = {
    'a': 1, 'b': 1, 'c': 0, 'd': 1, 'e': 1,
    'f': 0, 'g': 1, 'h': 0, 'i': 0, 'j': 0,
    'sad': 0, 'smiley': 0, 'xclaim': 0
}

eaders for the csv file and formatting fixes

In [ ]:
headers = ["label", "index", "nr_pix", "rows_with_1", "cols_with_1", "rows_with_3p",
           "cols_with_3p", "aspect_ratio", "neigh_1", "no_neigh_above", "no_neigh_below",
           "no_neigh_left", "no_neigh_right", "no_neigh_horiz", "no_neigh_vert",
           "connected_areas", "eyes", "density"]

In [ ]:
def format_features(features):
    features[5] = round(features[5], 1)  
    features[13] = int(np.asscalar(features[13])) 
    return features

calculating all the features

In [ ]:
def calculate_features(matrix, label_name):
    nr_pix = np.sum(matrix == 1)  
    rows_with_1 = sum(np.sum(row) == 1 for row in matrix)
    cols_with_1 = sum(np.sum(matrix[:, col]) == 1 for col in range(matrix.shape[1]))
    rows_with_3p = sum(np.sum(row) >= 3 for row in matrix)
    cols_with_3p = sum(np.sum(matrix[:, col]) >= 3 for col in range(matrix.shape[1]))

    #aspect ratio
    black_pixels = np.argwhere(matrix == 1)
    height = black_pixels[:, 0].max() - black_pixels[:, 0].min() + 1 if len(black_pixels) > 0 else 1
    width = black_pixels[:, 1].max() - black_pixels[:, 1].min() + 1 if len(black_pixels) > 0 else 1
    aspect_ratio = round(width / height, 1)
    

    #for neighbouring pixels
    padded = np.pad(matrix, pad_width=1, mode='constant', constant_values=0)
    shifts = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    neigh_1 = 0
    no_neigh_above = 0
    no_neigh_below = 0
    no_neigh_left = 0
    no_neigh_right = 0
    no_neigh_horiz = 0
    no_neigh_vert = 0
    for i in range(1, 19):
        for j in range(1, 19):
            if padded[i, j] == 1:
                neighbors = [padded[i + di, j + dj] for di, dj in shifts]
                count = sum(neighbors)
                if count == 1:
                    neigh_1 += 1
                if sum(neighbors[:3]) == 0:
                    no_neigh_above += 1
                if sum(neighbors[5:]) == 0:
                    no_neigh_below += 1
                if sum(neighbors[::3]) == 0:
                    no_neigh_left += 1
                if sum(neighbors[2::3]) == 0:
                    no_neigh_right += 1
                if sum(neighbors[2:5]) == 0:
                    no_neigh_horiz += 1
                if sum(neighbors[:2] + neighbors[5:]) == 0:
                    no_neigh_vert += 1

    #connected components
    structure = np.ones((3, 3), dtype=int)
    _, connected_areas = label(matrix, structure=structure)

    #hardcoded eye values
    eyes = label_to_eye_count.get(label_name, 0)
    
    #density calculation
    density = nr_pix / (height * width) if height * width > 0 else 0
    return [nr_pix, rows_with_1, cols_with_1, rows_with_3p, cols_with_3p, aspect_ratio,
            neigh_1, no_neigh_above, no_neigh_below, no_neigh_left, no_neigh_right,
            no_neigh_horiz, no_neigh_vert, connected_areas, eyes, density]

riting to file

In [ ]:
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(headers)  # Write header row
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".csv"):
            parts = filename[:-4].split("_")
            if len(parts) == 3:
                label_ = parts[1]
                index = parts[2]
                with open(os.path.join(folder_path, filename), "r") as f:
                    matrix = np.array([list(map(int, row)) for row in csv.reader(f)], dtype=int)
                
                features = calculate_features(matrix, label_)
                
                writer.writerow([label_, index] + features)
            else:
                print(f"Filename format issue: {filename}")